In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sqlite3
import os

db = sqlite3.connect(os.path.join('../', 'instance', 'flaskr.sqlite'), detect_types=sqlite3.PARSE_DECLTYPES)
db.row_factory = sqlite3.Row

In [7]:

%run ../py/html_analysis.py
ha = HeaderAnalysis()
ha.store_unique_list('supp_pay_nonheaders_list', '<li>401(k)</li>')
supp_pay_nonheaders_list = s.load_object('supp_pay_nonheaders_list')
for navigable_parent in supp_pay_nonheaders_list:
    db.execute('''
UPDATE NavigableParents
SET is_header = 0, is_supplemental_pay = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\supp_pay_nonheaders_list.pickle


In [9]:

sql_str = '''
SELECT navigable_parent
FROM NavigableParents
WHERE
    navigable_parent_id IN (
        SELECT navigable_parent_id
        FROM NavigableParentSequence
        WHERE file_name_id IN (
            SELECT file_name_id
            FROM NavigableParentSequence
            WHERE navigable_parent_id = (
                SELECT navigable_parent_id
                FROM NavigableParents
                WHERE navigable_parent = '<li>401(k)</li>'
                )
            )
        ) AND
    is_header IS NULL'''
cursor_obj = db.execute(sql_str)
row_objs_list = cursor_obj.fetchall()
child_strs_list = [row_obj['navigable_parent'] for row_obj in row_objs_list]

In [10]:

set(child_strs_list)

{'<p>· Masters or PhD Degree with a minimum of 2 years experience in at least one of the following: Machine Learning (Supervised, Unsupervised), Bayesian Statistics, Statistical Modeling (e.g. Survival Analysis, Generalized Linear models), Advanced Machine Learning (e.g. Reinforcement Learning, Deep Learning) , NLP, Dynamic Programming &amp; Optimal Control Theory</p>', ':', '<p>You will be joining a highly talented team of engineers and data scientists working on implementing production machine learning models in the media space. We are helping a large company optimize their production models and further improve accuracy by leveraging additional 3rd-party datasets. This role will require exploratory model testing, production system maintenance, and collaboration with technical SME to improve performance, accuracy, and methodology.</p>', '<li>At least 3 years experience developing web and server applications using Python, Java, C++, C#, or similar: Experience designing and implementing

In [8]:

subtypes_list = ['is_task_scope', 'is_minimum_qualification', 'is_preferred_qualification', 'is_legal_notification', 'is_job_title',
                 'is_office_location', 'is_job_duration', 'is_supplemental_pay', 'is_educational_requirement', 'is_interview_procedure',
                 'is_corporate_scope', 'is_posting_date', 'is_other']
for primary_column in subtypes_list:
    for secondary_column in subtypes_list:
        sql_str = f'''
UPDATE NavigableParents
SET {secondary_column} = 0
WHERE
    {primary_column} = 1 AND
    {secondary_column} IS NULL;'''
        db.execute(sql_str)
db.commit()

In [3]:

db.execute('DROP TABLE IF EXISTS FileNames;')
db.execute('''
CREATE TABLE FileNames(
    file_name_id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_name TEXT NOT NULL
);''')
db.commit()

In [7]:

%run ../py/html_analysis.py
ha = HeaderAnalysis()
files_list = os.listdir(ha.SAVES_HTML_FOLDER)
for file_name in files_list:
    if file_name.endswith('html'):
        db.execute('INSERT INTO FileNames (file_name) VALUES (?)', (file_name,))
db.commit()

In [9]:

db.execute('DROP TABLE IF EXISTS HeaderTags;')
db.execute('''
CREATE TABLE HeaderTags(
    header_tag_id INTEGER PRIMARY KEY AUTOINCREMENT,
    header_tag TEXT NOT NULL
);''')
db.commit()

In [14]:

tag_set = set()
for file_name in files_list:
    child_strs_list = ha.get_child_strs_from_file(file_name)
    child_tags_list = ha.get_child_tags_list(child_strs_list)
    for child_tag in child_tags_list:
        tag_set.add(child_tag)

In [10]:

for child_tag in sorted(tag_set):
    db.execute('INSERT INTO HeaderTags (header_tag) VALUES (?)', (child_tag,))
db.commit()

In [43]:

db.execute('DROP TABLE IF EXISTS NavigableParents;')
db.execute('''
CREATE TABLE NavigableParents (
    navigable_parent_id INTEGER PRIMARY KEY AUTOINCREMENT,
    navigable_parent TEXT NOT NULL,
    is_header BIT NULL,
    is_task_scope BIT NULL,
    is_minimum_qualification BIT NULL,
    is_preferred_qualification BIT NULL,
    is_legal_notification BIT NULL,
    is_job_title BIT NULL,
    is_office_location BIT NULL,
    is_job_duration BIT NULL,
    is_supplemental_pay BIT NULL,
    is_educational_requirement BIT NULL,
    is_interview_procedure BIT NULL,
    is_corporate_scope BIT NULL,
    is_posting_date BIT NULL,
    is_other BIT NULL,
    is_qualification BIT NULL
);''')
db.commit()

In [44]:

navigable_parent_set = set()
for file_name in files_list:
    child_strs_list = ha.get_child_strs_from_file(file_name)
    for child_str in child_strs_list:
        navigable_parent_set.add(child_str)

In [45]:

for child_str in sorted(navigable_parent_set):
    db.execute('INSERT INTO NavigableParents (navigable_parent) VALUES (?)', (child_str,))
db.commit()

In [46]:

db.execute('DROP TABLE IF EXISTS HeaderTagSequence;')
db.execute('''
CREATE TABLE HeaderTagSequence(
    header_tag_sequence_id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_name_id INTEGER NOT NULL,
    header_tag_id INTEGER NOT NULL,
    sequence_order INTEGER NOT NULL
);''')
db.commit()

In [47]:

for file_name in files_list:
    file_name_id = db.execute('SELECT file_name_id FROM FileNames WHERE file_name = ?', (file_name,)).fetchone()['file_name_id']
    child_strs_list = ha.get_child_strs_from_file(file_name)
    child_tags_list = ha.get_child_tags_list(child_strs_list)
    for sequence_order, header_tag in enumerate(child_tags_list):
        header_tag_id = db.execute('SELECT header_tag_id FROM HeaderTags WHERE header_tag = ?', (header_tag,)).fetchone()['header_tag_id']
        db.execute('INSERT INTO HeaderTagSequence (file_name_id, header_tag_id, sequence_order) VALUES (?, ?, ?)',
                   (file_name_id, header_tag_id, sequence_order))
db.commit()

In [48]:

db.execute('DROP TABLE IF EXISTS NavigableParentSequence;')
db.execute('''
CREATE TABLE NavigableParentSequence(
    navigable_parent_sequence_id INTEGER PRIMARY KEY AUTOINCREMENT,
    file_name_id INTEGER NOT NULL,
    navigable_parent_id INTEGER NOT NULL,
    sequence_order INTEGER NOT NULL
);''')
db.commit()

In [49]:

for file_name in files_list:
    file_name_id = db.execute('SELECT file_name_id FROM FileNames WHERE file_name = ?', (file_name,)).fetchone()['file_name_id']
    child_strs_list = ha.get_child_strs_from_file(file_name)
    for sequence_order, navigable_parent in enumerate(child_strs_list):
        navigable_parent_id = db.execute('SELECT navigable_parent_id FROM NavigableParents WHERE navigable_parent = ?',
                                         (navigable_parent,)).fetchone()['navigable_parent_id']
        db.execute('INSERT INTO NavigableParentSequence (file_name_id, navigable_parent_id, sequence_order) VALUES (?, ?, ?)',
                   (file_name_id, navigable_parent_id, sequence_order))
db.commit()

In [52]:

%run ../load_magic/storage.py
s = Storage()
[f's.{fn}' for fn in dir(s) if not fn.startswith('_')]

['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.pickle_exists', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']

In [60]:

corp_scope_headers_list = s.load_object('corp_scope_headers_list')
for navigable_parent in corp_scope_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_corporate_scope = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [64]:

NAVIGABLE_PARENT_IS_HEADER_DICT = s.load_object('NAVIGABLE_PARENT_IS_HEADER_DICT')
for navigable_parent, is_header in NAVIGABLE_PARENT_IS_HEADER_DICT.items():
    if is_header:
        db.execute('''
UPDATE NavigableParents
SET is_header = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
    else:
        db.execute('''
UPDATE NavigableParents
SET is_header = 0
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [66]:

NAVIGABLE_PARENT_IS_QUAL_DICT = s.load_object('NAVIGABLE_PARENT_IS_QUAL_DICT')
for navigable_parent, is_qualification in NAVIGABLE_PARENT_IS_QUAL_DICT.items():
    if is_qualification:
        db.execute('''
UPDATE NavigableParents
SET is_qualification = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
    else:
        db.execute('''
UPDATE NavigableParents
SET is_qualification = 0
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [69]:

task_scope_headers_list = s.load_object('task_scope_headers_list')
for navigable_parent in task_scope_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_task_scope = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [70]:

office_loc_headers_list = s.load_object('office_loc_headers_list')
for navigable_parent in office_loc_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_office_location = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [71]:

req_quals_headers_list = s.load_object('req_quals_headers_list')
for navigable_parent in req_quals_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_minimum_qualification = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [72]:

supp_pay_headers_list = s.load_object('supp_pay_headers_list')
for navigable_parent in supp_pay_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_supplemental_pay = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [73]:

preff_quals_headers_list = s.load_object('preff_quals_headers_list')
for navigable_parent in preff_quals_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_preferred_qualification = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [74]:

legal_notifs_headers_list = s.load_object('legal_notifs_headers_list')
for navigable_parent in legal_notifs_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_legal_notification = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [75]:

other_headers_list = s.load_object('other_headers_list')
for navigable_parent in other_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_other = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [76]:

educ_reqs_headers_list = s.load_object('educ_reqs_headers_list')
for navigable_parent in educ_reqs_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_educational_requirement = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [77]:

interv_proc_headers_list = s.load_object('interv_proc_headers_list')
for navigable_parent in interv_proc_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_interview_procedure = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [78]:

post_date_headers_list = s.load_object('post_date_headers_list')
for navigable_parent in post_date_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_posting_date = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [79]:

job_duration_headers_list = s.load_object('job_duration_headers_list')
for navigable_parent in job_duration_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_job_duration = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [80]:

job_title_headers_list = s.load_object('job_title_headers_list')
for navigable_parent in job_title_headers_list:
    db.execute('''
UPDATE NavigableParents
SET is_job_title = 1
WHERE (navigable_parent = ?)''', (navigable_parent,))
db.commit()

In [81]:

import random

pos_explanation_dict = s.load_object('pos_explanation_dict')
random.choice(list(pos_explanation_dict.items()))

('H-PD', 'Post Date Header')

In [87]:

db.execute('DROP TABLE IF EXISTS PartsOfSpeech;')
db.execute('''
CREATE TABLE PartsOfSpeech(
    pos_id INTEGER PRIMARY KEY AUTOINCREMENT,
    pos_symbol TEXT NOT NULL,
    pos_explanation TEXT NOT NULL
);''')
db.commit()

In [88]:

pos_explanation_dict = s.load_object('pos_explanation_dict')
for pos_symbol, pos_explanation in pos_explanation_dict.items():
    if pos_symbol.startswith('H-'):
        db.execute('INSERT INTO PartsOfSpeech (pos_symbol, pos_explanation) VALUES (?, ?)',
                   (pos_symbol, pos_explanation))
        db.execute('INSERT INTO PartsOfSpeech (pos_symbol, pos_explanation) VALUES (?, ?)',
                   (pos_symbol.replace('H-', 'O-'), pos_explanation.replace(' Header', ' Non-header')))
db.commit()


----

In [ ]:

from flask import Flask

[f'Flask.{fn}' for fn in dir(Flask) if not fn.startswith('_')]

In [ ]:

Flask?

In [5]:

__name__

'__main__'

In [ ]:

app = Flask(__name__, instance_relative_config=True)
[f'app.{fn}' for fn in dir(app) if not fn.startswith('_')]

In [23]:

[f'app.{fn}' for fn in dir(app) if 'config' in fn.lower()]

['app.config', 'app.config_class', 'app.default_config', 'app.make_config']

In [27]:

config_obj = app.config
[f'config_obj.{fn}' for fn in dir(config_obj) if not fn.startswith('_')]

['config_obj.clear', 'config_obj.copy', 'config_obj.from_envvar', 'config_obj.from_json', 'config_obj.from_mapping', 'config_obj.from_object', 'config_obj.from_pyfile', 'config_obj.fromkeys', 'config_obj.get', 'config_obj.get_namespace', 'config_obj.items', 'config_obj.keys', 'config_obj.pop', 'config_obj.popitem', 'config_obj.root_path', 'config_obj.setdefault', 'config_obj.update', 'config_obj.values']

In [29]:

config_obj = app.config
for k, v in config_obj.items():
    print(f'{k}: {v}')

ENV: production
DEBUG: False
TESTING: False
PROPAGATE_EXCEPTIONS: None
PRESERVE_CONTEXT_ON_EXCEPTION: None
SECRET_KEY: None
PERMANENT_SESSION_LIFETIME: 31 days, 0:00:00
USE_X_SENDFILE: False
SERVER_NAME: None
APPLICATION_ROOT: /
SESSION_COOKIE_NAME: session
SESSION_COOKIE_DOMAIN: None
SESSION_COOKIE_PATH: None
SESSION_COOKIE_HTTPONLY: True
SESSION_COOKIE_SECURE: False
SESSION_COOKIE_SAMESITE: None
SESSION_REFRESH_EACH_REQUEST: True
MAX_CONTENT_LENGTH: None
SEND_FILE_MAX_AGE_DEFAULT: 12:00:00
TRAP_BAD_REQUEST_ERRORS: None
TRAP_HTTP_EXCEPTIONS: False
EXPLAIN_TEMPLATE_LOADING: False
PREFERRED_URL_SCHEME: http
JSON_AS_ASCII: True
JSON_SORT_KEYS: True
JSONIFY_PRETTYPRINT_REGULAR: False
JSONIFY_MIMETYPE: application/json
TEMPLATES_AUTO_RELOAD: None
MAX_COOKIE_SIZE: 4093


In [30]:

[f'app.{fn}' for fn in dir(app) if 'blue' in fn.lower()]

['app._blueprint_order', 'app.blueprints', 'app.iter_blueprints', 'app.register_blueprint']

In [33]:

app.register_blueprint?

Signature: app.register_blueprint(blueprint, **options)
Docstring:
Register a :class:`~flask.Blueprint` on the application. Keyword
arguments passed to this method will override the defaults set on the
blueprint.

Calls the blueprint's :meth:`~flask.Blueprint.register` method after
recording the blueprint in the application's :attr:`blueprints`.

:param blueprint: The blueprint to register.
:param url_prefix: Blueprint routes will be prefixed with this.
:param subdomain: Blueprint routes will match on this subdomain.
:param url_defaults: Blueprint routes will use these default values for
    view arguments.
:param options: Additional keyword arguments are passed to
    :class:`~flask.blueprints.BlueprintSetupState`. They can be
    accessed in :meth:`~flask.Blueprint.record` callbacks.

.. versionadded:: 0.7
File:      c:\users\dev\documents\repositories\job-hunting\jh\lib\site-packages\flask\app.py
Type:      method
